In [ ]:
pip install torch torchvision pillow

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt

#Setting
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


class SingleImageDataset(Dataset):
    def __init__(self, cat_path, dog_path, transform=None):
        self.transform = transform
        self.cat_img = Image.open(cat_path).convert("RGB")
        self.dog_img = Image.open(dog_path).convert("RGB")
        self.labels = [0, 1]  # 0 for cat, 1 for dog
        
    def __len__(self):
        return 2  # We have 2 samples
    
    def __getitem__(self, idx):
        if idx == 0:
            img = self.cat_img
        else:
            img = self.dog_img
        if self.transform: 
            img = self.transform(img)
        return img, self.labels[idx]

# Define th e transforms
transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


CAT_PATH = "/Users/ryanhengboonchoon/Downloads/51vWTX+4VkL._AC_UF1000,1000_QL80_.jpg"
DOG_PATH = "/Users/ryanhengboonchoon/Downloads/Labrador_Retriever_portrait.jpg"

#Create dataset and dataloader
dataset = SingleImageDataset(CAT_PATH, DOG_PATH, transform=transform)
train_loader = DataLoader(dataset, batch_size=2, shuffle=True)

#CNN model
class PetClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),  # (100x100)
            nn.ReLU(),
            nn.MaxPool2d(2, 2),              # (50x50)
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),              # (25x25)
            nn.Dropout(0.2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(25*25*32, 128),
            nn.ReLU(),
            nn.Linear(128, 2)
        )
        
    def forward(self, x):
        x = self.conv_layers(x)
        return self.fc_layers(x)

#Initialize 
model = PetClassifier().to(device)

#Training setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#Training loop
num_epochs = 20
print("Starting training...")
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Track metrics
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
    
    #Print progress
    accuracy = correct / len(dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {total_loss/len(train_loader):.4f} Accuracy: {accuracy:.2f}")

#Prediction f(x)
def predict(image_path):
    # Load and transform image
    img = Image.open(image_path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0).to(device)
    
    #Make prediction
    model.eval()
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted = torch.max(output, 1)
        return "Cat" if predicted.item() == 0 else "Dog"

#Test prediction (replace with your test image path)
test_image = "/Users/ryanhengboonchoon/Downloads/cat_resized.jpg"
print(f"Prediction: {predict(test_image)}")

Using device: cpu
Starting training...
Epoch [1/20] Loss: 0.6997 Accuracy: 0.50
Epoch [2/20] Loss: 0.2853 Accuracy: 1.00
Epoch [3/20] Loss: 0.0266 Accuracy: 1.00
Epoch [4/20] Loss: 0.0013 Accuracy: 1.00
Epoch [5/20] Loss: 0.0000 Accuracy: 1.00
Epoch [6/20] Loss: 0.0000 Accuracy: 1.00
Epoch [7/20] Loss: 0.0000 Accuracy: 1.00
Epoch [8/20] Loss: 0.0000 Accuracy: 1.00
Epoch [9/20] Loss: 0.0000 Accuracy: 1.00
Epoch [10/20] Loss: 0.0000 Accuracy: 1.00
Epoch [11/20] Loss: 0.0000 Accuracy: 1.00
Epoch [12/20] Loss: 0.0000 Accuracy: 1.00
Epoch [13/20] Loss: 0.0000 Accuracy: 1.00
Epoch [14/20] Loss: 0.0000 Accuracy: 1.00
Epoch [15/20] Loss: 0.0000 Accuracy: 1.00
Epoch [16/20] Loss: 0.0000 Accuracy: 1.00
Epoch [17/20] Loss: 0.0000 Accuracy: 1.00
Epoch [18/20] Loss: 0.0000 Accuracy: 1.00
Epoch [19/20] Loss: 0.0000 Accuracy: 1.00
Epoch [20/20] Loss: 0.0000 Accuracy: 1.00
Prediction: Cat
